In [1]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import random
import yaml

from utility import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

# paths
model_outputs_dir = params['model_dir']
skims_dir = _join(model_outputs_dir, "skims")
summary_outputs = params['summary_dir']
concept_id = params['concept_id']
ctramp_dir = params['ctramp_dir']
iteration = params['iteration']

concept_id = params['concept_id']
time_period_mapping = params['time_periods_mapping']
link21_purp_mapping = params['purpose_mapping']
mode_cat_mapping = params['mode_mapping']
time_periods = params['periods']
acc_egg_modes = params['access_egress_modes']
period = params['periods']

preprocess_dir = _join(ctramp_dir, '_pre_process_files')
perf_measure_columns = params['final_columns']
summary_dir = params['summary_dir']

demand_matrices_dir = _join(model_outputs_dir, "demand_matrices")
transit_demand_dir = _join(demand_matrices_dir, "transit")
transit_skims_dir = _join(skims_dir, "transit")
best_path_skim_dir = params['best_path_skim_dir']

In [3]:
df_trips = pd.read_parquet(_join(preprocess_dir, 'trip_roster.parquet'))

In [4]:
df_trn = df_trips.loc[df_trips['trip_mode'].isin([6,7,8])]

In [5]:
create_rail_crowding_od_pairs(preprocess_dir, transit_skims_dir, period, acc_egg_modes)

Period:  am
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR
Period:  md
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR
Period:  pm
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR
Period:  ev
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR
Period:  ea
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR


In [6]:
df_temp = []

for period in time_periods:
    print(f'processing - {period}')
    
    df_od_pr = omx.open_file(_join(preprocess_dir, "rail_od_v9_trim_" + period.upper() + ".omx"))
    df_od_cwd = omx.open_file(_join(preprocess_dir, "rail_crowding_od_v9_trim_" + period.upper() + ".omx"))
    df_trn_pd = df_trn[df_trn['Period'] == period]
    
    #walk transit
    df_trn_acc = df_trn_pd[df_trn_pd['Mode'] == 'WALK_TRANSIT']
    df_rail_od = skim_core_to_df(df_od_pr, 'WLK_TRN_WLK')
    df_rail_cwd = skim_core_to_df(df_od_cwd, 'WLK_TRN_WLK', cols =['orig', 'dest', 'crowd']) # add walk access/egress
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_wlk = pd.merge(df_trn_acc, df_rail_od, 
                          left_on =['orig_taz', 'dest_taz'], 
                          right_on=['orig', 'dest'], 
                          how ='inner')
    df_trn_wlk = pd.merge(df_trn_wlk, df_rail_cwd,
                          left_on =['orig_taz', 'dest_taz'], 
                          right_on=['orig', 'dest'], 
                          how ='inner')
    
    
    # PNR Transit
    df_trn_acc = df_trn_pd[df_trn_pd['Mode'] == 'PNR_TRANSIT']
    df_trn_acc_inbnd = df_trn_acc[df_trn_acc['inbound'] == 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'WLK_TRN_PNR')
    df_rail_cwd = skim_core_to_df(df_od_cwd, 'WLK_TRN_PNR', cols =['orig', 'dest', 'crowd']) # add walk access/egress
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_pnr_inb = pd.merge(df_trn_acc_inbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')
    df_trn_pnr_inb = pd.merge(df_trn_pnr_inb, df_rail_cwd, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')

    df_trn_acc_outbnd = df_trn_acc[df_trn_acc['inbound'] != 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'PNR_TRN_WLK')
    df_rail_cwd = skim_core_to_df(df_od_cwd, 'PNR_TRN_WLK', cols =['orig', 'dest', 'crowd']) # add walk access/egress
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_pnr_outbnd = pd.merge(df_trn_acc_outbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')
    df_trn_pnr_outbnd = pd.merge(df_trn_pnr_outbnd, df_rail_cwd, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')

    df_trn_pnr = pd.concat([df_trn_pnr_inb, df_trn_pnr_outbnd], ignore_index=True)
    
    # KNR Transit
    df_trn_acc = df_trn_pd[df_trn_pd['Mode'] == 'KNR_TRANSIT']
    df_trn_acc_inbnd = df_trn_acc[df_trn_acc['inbound'] == 1] # returning home
    df_rail_cwd = skim_core_to_df(df_od_cwd, 'WLK_TRN_KNR', cols =['orig', 'dest', 'crowd']) # add walk access/egress
    df_rail_od = skim_core_to_df(df_od_pr, 'WLK_TRN_KNR')
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_knr_inb = pd.merge(df_trn_acc_inbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')
    df_trn_knr_inb = pd.merge(df_trn_knr_inb, df_rail_cwd, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')


    df_trn_acc_outbnd = df_trn_acc[df_trn_acc['inbound'] != 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'KNR_TRN_WLK')
    df_rail_cwd = skim_core_to_df(df_od_cwd, 'KNR_TRN_WLK', cols =['orig', 'dest', 'crowd']) # add walk access/egress
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_knr_outbnd = pd.merge(df_trn_acc_outbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')
    df_trn_knr_outbnd = pd.merge(df_trn_knr_outbnd, df_rail_cwd, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')

    df_trn_knr = pd.concat([df_trn_knr_inb, df_trn_knr_outbnd], ignore_index=True)
    
    df_trn_rail = pd.concat([df_trn_wlk, df_trn_pnr, df_trn_knr], ignore_index=True)
    df_temp.append(df_trn_rail)

df_trn_rail = pd.concat(df_temp)

processing - am
processing - md
processing - pm
processing - ev
processing - ea


In [7]:
df_trn_rail['trips'] = df_trn_rail['trips']*df_trn_rail['crowd']/100

In [8]:
df_trn_rail.loc[df_trn_rail['Period'] == 'pm']

,hh_id,person_id,inbound,orig_taz,dest_taz,depart_hour,trip_mode,sampleRate,trip_type,trips,...,pp_share,link21_trip_purp,Period,Mode,orig_x,dest_x,rail_od,orig_y,dest_y,crowd
0,1783974,4329125.0,0,82,80,15,6,1.0,INM,0.0,...,0.0,work,pm,WALK_TRANSIT,82,80,1.0,82,80,0.0
1,1947355,4597986.0,1,82,80,18,6,1.0,INM,0.0,...,0.0,work,pm,WALK_TRANSIT,82,80,1.0,82,80,0.0
2,1944404,4592507.0,1,82,80,17,6,1.0,INM,0.0,...,0.0,work,pm,WALK_TRANSIT,82,80,1.0,82,80,0.0
3,1947360,4597997.0,1,82,80,18,6,1.0,INM,0.0,...,0.0,work,pm,WALK_TRANSIT,82,80,1.0,82,80,0.0
4,1946665,4596718.0,1,82,80,17,6,1.0,INM,0.0,...,0.0,work,pm,WALK_TRANSIT,82,80,1.0,82,80,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314314,1009496,NaN,0,2586,615,17,8,1.0,JNT,0.0,...,100.0,othmaint,pm,KNR_TRANSIT,2586,615,1.0,2586,615,0.0
314315,1254454,NaN,0,2653,2308,15,8,1.0,JNT,0.0,...,0.0,shopping,pm,KNR_TRANSIT,2653,2308,1.0,2653,2308,0.0
314316,1315402,NaN,0,2656,2455,15,8,1.0,JNT,0.0,...,0.0,social,pm,KNR_TRANSIT,2656,2455,1.0,2656,2455,0.0
314317,1064748,NaN,0,2663,2530,18,8,1.0,JNT,0.0,...,0.0,othmaint,pm,KNR_TRANSIT,2663,2530,1.0,2663,2530,0.0


In [9]:
# summarise for prioirty population
#regional value
region_value = df_trn_rail.groupby(['Period'])['trips'].mean().reset_index()
region_value = region_value.rename(columns={'trips': 'Value'})
region_value['Concept_ID'] = concept_id
region_value['Metric_ID'] = 'A1.9'
region_value['Metric_name'] = 'Crowding (Region)'
region_value['Submetric'] = 'A1.9.1'
region_value['Description'] = 'Regional crowding level'
region_value['Population'] = 'Whole Population'
region_value['Geography'] = 'Region'
region_value['Origin_zone'] = ''
region_value['Dest_zone'] = ''
region_value['Purpose'] = ''
region_value['Zone_ID'] = ''
region_value['Units'] = 'minutes'
region_value['Total_Increment'] = ''
region_value = region_value[perf_measure_columns]

In [10]:
region_value

,Concept_ID,Metric_ID,Metric_name,Submetric,Description,Population,Period,Geography,Zone_ID,Origin_zone,Dest_zone,Purpose,Value,Units,Total_Increment
0,BaseYear2050,A1.9,Crowding (Region),A1.9.1,Regional crowding level,Whole Population,am,Regional,Megaregion,,,,8.744692,minutes,
1,BaseYear2050,A1.9,Crowding (Region),A1.9.1,Regional crowding level,Whole Population,ea,Regional,Megaregion,,,,1.373053,minutes,
2,BaseYear2050,A1.9,Crowding (Region),A1.9.1,Regional crowding level,Whole Population,ev,Regional,Megaregion,,,,0.000000,minutes,
3,BaseYear2050,A1.9,Crowding (Region),A1.9.1,Regional crowding level,Whole Population,md,Regional,Megaregion,,,,0.610478,minutes,
4,BaseYear2050,A1.9,Crowding (Region),A1.9.1,Regional crowding level,Whole Population,pm,Regional,Megaregion,,,,0.000000,minutes,


In [11]:
region_value.to_csv(_join(summary_dir, 'regional_crowding_A1.9.csv'), index=False)

In [12]:
trn_skm = omx.open_file(_join(transit_skims_dir, "trnskmmd_WLK_TRN_WLK.omx"))

In [13]:
np.array(trn_skm['CROWD'])

array([[  0.      , 199.6233  ,  58.984287, ..., 312.40338 , 291.67355 ,
        312.40338 ],
       [ 27.206871,   0.      ,  50.51666 , ..., 374.5849  , 375.1736  ,
        374.5849  ],
       [ 67.497215,  61.949657,   0.      , ...,  45.52742 ,  47.041206,
         45.52742 ],
       ...,
       [330.18365 , 447.4546  ,  45.65552 , ...,   0.      ,   0.      ,
          0.      ],
       [330.18365 , 447.4546  ,  45.65552 , ...,   0.      ,   0.      ,
          0.      ],
       [330.18365 , 447.4546  ,  45.65552 , ...,   0.      ,   0.      ,
          0.      ]], dtype=float32)

In [14]:
period

'ea'

In [15]:
transit_skims_dir = r'C:\MTC_tmpy\TM2_Run03\calibration_3332\skims'

In [16]:
for per in period:
    print("Period: ", per)
    for acc_egg in acc_egg_modes:
        #print("Access Egress Mode: ",acc_egg)
        trn_skm = omx.open_file(_join(transit_skims_dir, "trnskm" + per.lower() +"_" + acc_egg.upper() + ".omx"))
        crowd_summary = np.array(trn_skm['CROWD']).sum()
        print(f'{per} : {acc_egg} : {crowd_summary}')

Period:  e


OSError: ``C:\MTC_tmpy\TM2_Run03\calibration_3332\skims\trnskme_WLK_TRN_WLK.omx`` does not exist

#county 
county_df_temp = df_trn_rail.groupby(['orig_county', 'dest_county', 'Period'])['trips'].mean().reset_index()
county_df_temp = county_df_temp.rename(columns={'trips': 'Value',
                                               'orig_county': 'Origin_zone',
                                               'dest_county': 'Dest_zone'})
county_df_temp['Concept_ID'] = concept_id
county_df_temp['Metric_ID'] = 'A1.9'
county_df_temp['Metric_name'] = 'Crowding (Region)'
county_df_temp['Submetric'] = 'A1.9.2'
county_df_temp['Description'] = 'Crowding level between origin and destination county'
county_df_temp['Population'] = 'Whole Population'
county_df_temp['Geography'] = 'County'
county_df_temp['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
county_df_temp['Units'] = 'minutes'
county_df_temp['Total_Increment'] = ''


#super district
sd_df_temp = df_trn_rail.groupby(['orig_super_dist', 'dest_super_dist', 'Period'])['trips'].mean().reset_index()
sd_df_temp = sd_df_temp.rename(columns={'trips': 'Value',
                                        'orig_super_dist': 'Origin_zone',
                                        'dest_super_dist': 'Dest_zone'})
sd_df_temp['Concept_ID'] = concept_id
sd_df_temp['Metric_ID'] = 'A1.9'
sd_df_temp['Metric_name'] = 'Crowding (Region)'
sd_df_temp['Submetric'] = 'A1.9.3'
sd_df_temp['Description'] = 'Regional crowding level'
sd_df_temp['Population'] = 'Whole Population'
sd_df_temp['Geography'] = 'Super district'
sd_df_temp['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
sd_df_temp['Units'] = 'minutes'
sd_df_temp['Total_Increment'] = ''


#RDM Zones
rdm_df_temp = df_trn_rail.groupby(['orig_rdm_zones', 'dest_rdm_zones', 'Period'])['trips'].mean().reset_index()
rdm_df_temp = rdm_df_temp.rename(columns={'trips': 'Value',
                                        'orig_rdm_zones': 'Origin_zone',
                                        'dest_rdm_zones': 'Dest_zone'})
rdm_df_temp['Concept_ID'] = concept_id
rdm_df_temp['Metric_ID'] = 'A1.9'
rdm_df_temp['Metric_name'] = 'Crowding (Region)'
rdm_df_temp['Submetric'] = 'A1.9.4'
rdm_df_temp['Description'] = 'Regional crowding level'
rdm_df_temp['Population'] = 'Whole Population'
rdm_df_temp['Geography'] = 'RDM'
rdm_df_temp['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
rdm_df_temp['Units'] = 'minutes'
rdm_df_temp['Total_Increment'] = ''




# transbay region
tb_value = df_trn_rail[df_trn_rail['transbay_od']==1]
tb_value = tb_value.groupby(['Period'])['trips'].mean().reset_index()

tb_value['Concept_ID'] = concept_id
tb_value['Metric_ID'] = 'A1.9'
tb_value['Metric_name'] = 'Crowding (Region)'
tb_value['Submetric'] = 'A1.9.5'
tb_value['Description'] = 'regional crowding level'
tb_value['Population'] = 'Whole Population'
tb_value['Geography'] = 'Transbay'
tb_value['Zone_ID'] = ''
tb_value['Origin_zone'] = ''
tb_value['Dest_zone'] = ''
tb_value['Units'] = 'minutes'
tb_value['Total_Increment'] = ''